In [2]:
#Loading libraries 
import pandas as pd
import numpy as np
import random as rnd
from collections import Counter # value count calculator
from sklearn.model_selection import train_test_split

In [3]:
#pick the dataset

df = pd.read_csv(r'diabetic_data.csv')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
print(f'Number of rows: {len(df)}, Number of columns: {len(df.columns)}')
#Displaying first 10 rows of data
df.head(10).T

Number of rows: 101766, Number of columns: 50


,0,1,2,3,4,5,6,7,8,9
encounter_id,2278392,149190,64410,500364,16680,35754,55842,63768,12522,15738
patient_nbr,8222157,55629189,86047875,82442376,42519267,82637451,84259809,114882984,48330783,63555939
race,Caucasian,Caucasian,AfricanAmerican,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian,Caucasian
gender,Female,Female,Female,Male,Male,Male,Male,Male,Female,Female
age,[0-10),[10-20),[20-30),[30-40),[40-50),[50-60),[60-70),[70-80),[80-90),[90-100)
weight,?,?,?,?,?,?,?,?,?,?
admission_type_id,6,1,1,1,1,2,3,1,2,3
discharge_disposition_id,25,1,1,1,1,1,1,1,1,3
admission_source_id,1,7,7,7,7,2,2,7,4,4
time_in_hospital,1,3,2,2,1,3,4,5,13,12


In [5]:
#Removing duplicate records leaving only earliest encounter_id
df = df.sort_values(['patient_nbr', 'encounter_id']).drop_duplicates(subset= ['patient_nbr'], keep = 'first')

In [6]:
# col_distinct_val_check = [col for col in df.columns if col not in ['encounter_id', 'patient_nbr']]

# for col in col_distinct_val_check:
#     print(col)
#     print(set(df[col]))
#     print(input("Assess distinct values and press Enter"))

In [7]:
for col in df.columns:
    if df[col].dtype == object:
        proportion_null = df[col].isna().sum()/len(df)
        #proportion_null = df[col][df[col] == '?'].count()/len(df)*100
        print(col, f'{proportion_null * 100:.2f}%')

race 0.00%
gender 0.00%
age 0.00%
weight 0.00%
payer_code 0.00%
medical_specialty 0.00%
diag_1 0.00%
diag_2 0.00%
diag_3 0.00%
max_glu_serum 0.00%
A1Cresult 0.00%
metformin 0.00%
repaglinide 0.00%
nateglinide 0.00%
chlorpropamide 0.00%
glimepiride 0.00%
acetohexamide 0.00%
glipizide 0.00%
glyburide 0.00%
tolbutamide 0.00%
pioglitazone 0.00%
rosiglitazone 0.00%
acarbose 0.00%
miglitol 0.00%
troglitazone 0.00%
tolazamide 0.00%
examide 0.00%
citoglipton 0.00%
insulin 0.00%
glyburide-metformin 0.00%
glipizide-metformin 0.00%
glimepiride-pioglitazone 0.00%
metformin-rosiglitazone 0.00%
metformin-pioglitazone 0.00%
change 0.00%
diabetesMed 0.00%
readmitted 0.00%


In [8]:
df['race'].value_counts(dropna=False)

Caucasian          53491
AfricanAmerican    12887
?                   1948
Hispanic            1517
Other               1178
Asian                497
Name: race, dtype: int64

In [9]:
df['race'] = df.race.fillna('Other')

In [10]:
df['gender'].value_counts(dropna=False)

Female             38025
Male               33490
Unknown/Invalid        3
Name: gender, dtype: int64

In [11]:
exclude_indexes = df[df['gender'] == 'Unknown/Invalid'].index.tolist()

exclude_indexes = exclude_indexes + df[df['diag_1'].isna()].index.tolist()

exclude_indexes = exclude_indexes + df[df['diag_2'].isna()].index.tolist()

exclude_indexes = exclude_indexes + df[df['diag_3'].isna()].index.tolist()

required_indexes = [index for index in df.index.tolist() if index not in list(set(exclude_indexes))]

df = df.loc[required_indexes]

In [12]:
df = df.drop(['weight','payer_code','medical_specialty','examide', 'citoglipton', 'glimepiride-pioglitazone'], axis = 1)

In [13]:
df['race'] = df['race'].astype('category').cat.codes

df['gender'] = df['gender'].replace('Male', 1)
df['gender'] = df['gender'].replace('Female', 0)

df['age'] = df['age'].replace('[0-10)', 0)
df['age'] = df['age'].replace('[10-20)', 1)
df['age'] = df['age'].replace('[20-30)', 2)
df['age'] = df['age'].replace('[30-40)', 3)
df['age'] = df['age'].replace('[40-50)', 4)
df['age'] = df['age'].replace('[50-60)', 5)
df['age'] = df['age'].replace('[60-70)', 6)
df['age'] = df['age'].replace('[70-80)', 7)
df['age'] = df['age'].replace('[80-90)', 8)
df['age'] = df['age'].replace('[90-100)', 9) 

In [14]:
df['diag_1'] = df['diag_1'].astype('category').cat.codes
df['diag_2'] = df['diag_2'].astype('category').cat.codes
df['diag_3'] = df['diag_3'].astype('category').cat.codes

In [15]:
#Lab Tests
df['A1Cresult'] = df['A1Cresult'].replace('>7', 2)
df['A1Cresult'] = df['A1Cresult'].replace('>8', 2)
df['A1Cresult'] = df['A1Cresult'].replace('Norm', 1)                            
df['A1Cresult'] = df['A1Cresult'].replace('None', 0, regex=True)

df['max_glu_serum'] = df['max_glu_serum'].replace('>200', 2)
df['max_glu_serum'] = df['max_glu_serum'].replace('>300', 2)
df['max_glu_serum'] = df['max_glu_serum'].replace('Norm', 1)
df['max_glu_serum'] = df['max_glu_serum'].replace('None', 0, regex=True)

In [16]:
medications = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'glipizide', \
'glyburide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'insulin', 'glyburide-metformin', \
'tolazamide','metformin-pioglitazone', 'metformin-rosiglitazone', 'glipizide-metformin','troglitazone', \
'tolbutamide', 'acetohexamide']

for col in medications:
    df[col] = df[col].replace('No', 0)
    df[col] = df[col].replace('Steady', 1)
    df[col] = df[col].replace('Up', 1)
    df[col] = df[col].replace('Down', 1)

In [17]:
df['change'] = df['change'].replace('Ch', 1)
df['change'] = df['change'].replace('No', 0)

df['diabetesMed'] = df['diabetesMed'].replace('Yes', 1)
df['diabetesMed'] = df['diabetesMed'].replace('No', 0)

df['readmitted'] = df['readmitted'].replace('<30', 1)
df['readmitted'] = df['readmitted'].replace('>30', 0)
df['readmitted'] = df['readmitted'].replace('NO', 0)

In [18]:
print(f'Number of rows: {len(df)}, Number of columns: {len(df.columns)}')

Number of rows: 71515, Number of columns: 44


In [19]:
features = ['race', 'gender', 'admission_type_id', 'discharge_disposition_id','admission_source_id', 'diag_1',\
'diag_2', 'diag_3', 'max_glu_serum', 'A1Cresult','metformin', 'repaglinide', 'nateglinide', 'chlorpropamide',\
'glimepiride', 'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', \
'acarbose', 'miglitol', 'troglitazone', 'tolazamide', 'insulin', 'glyburide-metformin', 'glipizide-metformin',\
'metformin-rosiglitazone', 'metformin-pioglitazone', 'change', 'diabetesMed']

x = df[features].iloc[:10000].copy()

y = df['readmitted'].iloc[:10000].copy()

x_train, x_test, y_train, y_test = train_test_split(x, y,random_state = 42,test_size = 0.3, train_size = 0.7)

In [20]:
 #CART (classification and regression tree
class Tree(): 
    """
    This class creates decision tree nodes
    """
    def __init__(self, Y: list, X: pd.DataFrame, min_samples_split = None, max_depth = None, depth = None,): 
        self.Y = Y 
        self.X = X
        
        self.min_samples_split = min_samples_split if min_samples_split else 10
        self.max_depth = max_depth if max_depth else 10

        self.depth = depth if depth else 0

        self.features = list(self.X.columns)

        self.counts = Counter(Y)
        
        # GINI impurity
        self.gini_impurity = self.get_GINI()

        # Sorting the counts and saving the final prediction of the node 
        counts_sorted = list(sorted(self.counts.items(), key=lambda item: item[1]))
        
#         if len(Y) == 0:
#             print("A node has 0 smaples - prediction will be None")
        
        # Getting the last item
        yhat = None
        if len(counts_sorted) > 0:
            yhat = counts_sorted[-1][0]

        # Saving to object attribute. This node will predict the class with the most frequent class
        self.yhat = yhat 

        # Saving the number of observations in the node 
        self.num_observations = len(Y)

        # Empty left and right nodes
        self.left_node = None 
        self.right_node = None 

        # Default values for splits
        self.best_feature = None 
        self.best_value = None      

    @staticmethod
    def GINI_impurity(y1_count: int, y2_count: int) -> float:
        """
        Calculate the GINI impurity
        """
        # Ensuring the correct types
        if y1_count is None:
            y1_count = 0

        if y2_count is None:
            y2_count = 0

        # total observations
        total_observ = y1_count + y2_count
        
        # When total observations is 0 then gini impurity 0
        if total_observ == 0:
            return 0.0

        # Getting the probability to see each of the classes
        p1 = y1_count / total_observ
        p2 = y2_count / total_observ
        
        # Calculating GINI 
        gini = 1 - (p1 ** 2 + p2 ** 2)
        # Returning the gini impurity
        return gini

    @staticmethod
    def rolling_average(x: np.array, window: int) -> np.array:
        """
        Calculates the rolling average 
        """
        return np.convolve(x, np.ones(window), 'valid') / window

    def get_GINI(self):
        """
        Calculate GINI impurity of a node 
        """
        # Getting the 0 and 1 counts
        y1_count, y2_count = self.counts.get(0, 0), self.counts.get(1, 0)

        # Getting the GINI impurity
        return self.GINI_impurity(y1_count, y2_count)

    def best_split(self) -> tuple:
        """
        Given the X features and Y targets calculates the best split 
        for a decision tree
        """
        # Creating a dataset for spliting
        df = self.X.copy()
        df['Y'] = self.Y

        # Getting the GINI impurity for the base input 
        GINI_base = self.get_GINI()

        # Finding which split yields the best GINI gain 
        max_gain = -1

        # Default best feature and split
        best_feature = None
        best_value = None

        for feature in self.features:
            # Droping missing values
            Xdf = df.dropna().sort_values(feature)
            
            # Sorting the values and getting the rolling average
            xmeans = self.rolling_average(Xdf[feature].unique(), 2)
            
            for value in xmeans:
                # Spliting the dataset 
                left_counts = Counter(Xdf[Xdf[feature]<value]['Y'])
                right_counts = Counter(Xdf[Xdf[feature]>=value]['Y'])

                # Getting the Y distribution from the dicts
                y0_left, y1_left, y0_right, y1_right = left_counts.get(0, 0), left_counts.get(1, 0), \
                right_counts.get(0, 0), right_counts.get(1, 0)

                # Getting the left and right gini impurities
                gini_left = self.GINI_impurity(y0_left, y1_left)
                gini_right = self.GINI_impurity(y0_right, y1_right)

                # Getting the obs count from the left and the right data splits
                n_left = y0_left + y1_left
                n_right = y0_right + y1_right

                # Calculating the weights for each of the nodes
                w_left = n_left / (n_left + n_right)
                w_right = n_right / (n_left + n_right)

                # Calculating the weighted GINI impurity
                wGINI = w_left * gini_left + w_right * gini_right

                # Calculating the GINI gain 
                GINIgain = GINI_base - wGINI

                # Checking if this is the best split so far 
                if GINIgain > max_gain:
                    best_feature = feature
                    best_value = value 
                    
                    # Setting the best gain to the current one 
                    max_gain = GINIgain
        return (best_feature, best_value)

    
    def fit(self):
        """
        Recursive function to create the decision tree
        """
        # Making a df from the data 
        df = self.X.copy()
        df['Y'] = self.Y

        # If there is GINI to be gained, we split further 
        if (self.depth < self.max_depth) and (self.num_observations >= self.min_samples_split):

            # Getting the best split 
            best_feature, best_value = self.best_split()
            
            if best_feature is not None:
                # Saving the best split to the current node 
                self.best_feature = best_feature
                self.best_value = best_value
                
                # Getting the left and right nodes
                left_df, right_df = df[df[best_feature]<=best_value].copy(), df[df[best_feature]>best_value].copy()
                # Creating the left and right nodes
                

                left_node = Tree(
                    left_df['Y'].values.tolist(), 
                    left_df[self.features], 
                    depth=self.depth + 1, 
                    max_depth=self.max_depth, 
                    min_samples_split=self.min_samples_split, 
                    )

                self.left_node = left_node 
                self.left_node.fit()

                right_node = Tree(
                    right_df['Y'].values.tolist(), 
                    right_df[self.features], 
                    depth=self.depth + 1, 
                    max_depth=self.max_depth, 
                    min_samples_split=self.min_samples_split,
                    )

                self.right_node = right_node
                self.right_node.fit()

    def predict(self, X:pd.DataFrame):
        predictions = []

        for _, x in X.iterrows():
            values = {}
            for feature in self.features:
                values.update({feature: x[feature]})
        
            predictions.append(self.predict_obs(values))
        
        return predictions

    def predict_obs(self, values: dict) -> int:
        """
        Predict the class given a set of features
        """
        this_node = self
        
        while this_node.depth < this_node.max_depth:
            # Traversing the nodes all the way to the bottom
            best_feature = this_node.best_feature
            best_value = this_node.best_value
            
            if this_node.num_observations < this_node.min_samples_split:
                break 
            
            if (values.get(best_feature) < best_value):
                if self.left_node is not None:
                    this_node = this_node.left_node
            else:
                if self.right_node is not None:
                    this_node = this_node.right_node
            
        return this_node.yhat

In [21]:
min_sample_split = [100,50,20,10,5]
max_depth = [1,3,5,10,20]

best_custom_dt_accuracy = 0
best_min_sample_split = 0
best_max_depth = 0 

for min_sample in min_sample_split:
    for m_depth in max_depth:
        gini_tree = Tree(y_train.iloc[:300], x_train.iloc[:300], min_sample, m_depth)
        gini_tree.fit()
        dt_gini_prediction = gini_tree.predict(x_test)
        dt_gini_correct_count = (dt_gini_prediction == np.array(y_test)).sum()
        custom_dt_accuracy = round((dt_gini_correct_count / len(y_test)*100), 3)
        print(f"Min sample split {min_sample}, Max depth {m_depth}, DT accuracy - {custom_dt_accuracy}%")
        
        if custom_dt_accuracy > best_custom_dt_accuracy:
                    best_custom_dt_accuracy = custom_dt_accuracy
                    best_min_sample_split = min_sample
                    best_max_depth = m_depth
                    
print(f'Best accuracy {best_custom_dt_accuracy}')
print(f'Best min sample split {best_min_sample_split}')
print(f'Best max tree depth {best_max_depth}')

Min sample split 100, Max depth 1, DT accuracy - 91.0%
Min sample split 100, Max depth 3, DT accuracy - 91.0%
Min sample split 100, Max depth 5, DT accuracy - 91.0%
Min sample split 100, Max depth 10, DT accuracy - 91.0%
Min sample split 100, Max depth 20, DT accuracy - 91.0%
Min sample split 50, Max depth 1, DT accuracy - 91.0%
Min sample split 50, Max depth 3, DT accuracy - 91.0%
Min sample split 50, Max depth 5, DT accuracy - 91.0%
Min sample split 50, Max depth 10, DT accuracy - 91.0%
Min sample split 50, Max depth 20, DT accuracy - 91.0%
Min sample split 20, Max depth 1, DT accuracy - 91.0%
Min sample split 20, Max depth 3, DT accuracy - 91.0%
Min sample split 20, Max depth 5, DT accuracy - 90.6%
Min sample split 20, Max depth 10, DT accuracy - 90.067%
Min sample split 20, Max depth 20, DT accuracy - 90.067%
Min sample split 10, Max depth 1, DT accuracy - 91.0%
Min sample split 10, Max depth 3, DT accuracy - 91.0%
Min sample split 10, Max depth 5, DT accuracy - 90.6%
Min sample sp

Custom Decision Tree accuracy 91.633%


In [25]:
class MyPerceptron():
    def __init__(
        self, 
        max_iter = 100, #Maximum of iteration of 100% accuracy is not achieved
        num_feat = 32 #Number of features in dataset
    ):
        self.max_iter = max_iter
        self.h = np.random.random(num_feat + 1) #hypothesis contains random  weights values for each feature and bias (+1)
#         self.h = np.array(np.zeros(num_feat+1), dtype=float)
        self.best_h = None
    
    def compute_linear_score_with_(self, X, h):
        s = None
        s = (X * h[:-1]).sum(axis=1) + h[-1]
        return s
    
    def predict_with_(self, X, h):
        return np.sign(self.compute_linear_score_with_(X, h)).astype(int)
    
    def predict(self, X):
        return self.predict_with_(X, self.best_h)
    
    def fit(self, X, y):
        iteration = 1
        min_error = 0
        while True:
            # predict using the current h
            predicted = self.predict_with_(X, self.h)
            # find errors
            error_indexes = np.nonzero(predicted != y)[0]
            # `nonzero` returns the indexes for multiple dimension array,
            # Here only the indexes of the first (and only) dimension is 
            # concerned, therefore the [0]
            # error indexes where prediction was wrong   

           
            if iteration == 1:
                min_error = len(error_indexes)
                self.best_h = self.h
            else:
                if len(error_indexes) < min_error:
                    min_error = len(error_indexes)
                    self.best_h = self.h

            if len(error_indexes) > 0:        
                rand = np.random.randint(len(error_indexes))
                i = error_indexes[rand]
                # i is a random index where prediction was wrong


                self.h[:-1] += X[i] * float(y[i]) #Weigths update
                
                # To update the weights
                # for example where prediction was wrong X = [4.6 3.4] and y = -1
                # 1st multiply [4.6 3.4] by -1 = [-4.6 -3.4]
                # 2nd add [-4.6 -3.4] to h which is [0. 1.] in first interation
                # result = [-4.6 -2.4] these are updated weigthts for second iteration

                # To update the bias
                # This is equivalent to using the homogeneous representation of x, where the last element is one

                self.h[-1] += float(y[i])
                # For example in the first interation bias = 0, and as all prediction 1 the any random wrong prediction -1
                # so we add 0 to -1 and second iteration bias = -1
                iteration += 1
                if iteration >= self.max_iter:
                    break
            else:
                print(f"{iteration} Train errors: {len(error_indexes)}")
                break # All the predictions are correct on the training data and there is nothing to adjust.

In [26]:
#Converting dataframes to arrays for perceptron
x_train_array = np.array(x_train)

y_train_array = np.array(y_train)
y_train_array[y_train_array == 0]  = -1

x_test_array = np.array(x_test)
y_test_array = np.array(y_test)

In [27]:
#Assessing perceptron performance
perp = MyPerceptron()
perp.fit(x_train_array, y_train_array)
perp_prediction = perp.predict(x_test_array)
perp_prediction[perp_prediction == -1]  = 0
# print(list(perp_prediction))
perp_correct_count = (perp_prediction == y_test_array).sum()

custom_perc_accuracy = round((perp_correct_count / len(y_test_array)*100),3)
print(f"Custom Perceptron accuracy {custom_perc_accuracy}%")

Custom Perceptron accuracy 91.533%


In [28]:
adaboost_model = {
    "weights": [],
    "weak_predictors": []}
rng  = np.random.RandomState(42)

max_iter = 100
iter_num = 0

sample_count = len(x_train)

w = np.ones(sample_count) / sample_count #Weights in first iteration = 1/ number of samples

ind = list(range(sample_count))

while True:
    
    resample_ind = sorted(rng.choice(
            ind, size = (sample_count,), replace = True, p=w)) 
            # It is necessary to generate a random index set,
            # because we need to sample both the X- and y-array
            # and need to keep the (X-y) correspondence consistent

    x_resampled = x_train.iloc[resample_ind] 
    y_resampled = y_train.iloc[resample_ind]
    
    #converting to array
    x_resampled_array = np.array(x_resampled)
    y_resampled_array = np.array(y_resampled)
    y_resampled_array[y_resampled_array == 0]  = -1
    
    g = MyPerceptron()
    g.fit(x_resampled_array, y_resampled_array)
    
    # Calculate the classifier weight
    err = (g.predict(x_resampled_array) != y_resampled_array).astype(float)
    # print(f'error calcualtion {err}')
    e = max(np.sum(err) / sample_count, 1e-6) 
    # max(..., 1e-6) is to ensure
    # the following quotient is numerically safely defined.
    # To calculate the weight for this weak classifier
    al = 1/2 * np.log((1 - e) / e)
    
    # Store the information of the model
    adaboost_model["weights"].append(al)
    adaboost_model["weak_predictors"].append(g)    
    
    # Re-weight the samples for the next step resampling
    x_train_array = np.array(x_train)
    y_train_array = np.array(y_train)
    y_train_array[y_train_array == 0]  = -1

    err_0 = (g.predict(x_train_array) != y_train_array).astype(float)
    # To compute the sample weights for the next round of iteration
    #   err_0 = [0, 1, 1, 0, 0, ...]
    #   is a 0/1 (float type) array with elements being 1.0 for 
    #   those samples on which the weak classifier has been wrong
    dw = np.exp(al * (err_0 - 0.5) * 2)

    w = dw * w
    w = w / w.sum()

    # Iteration update
    iter_num += 1
    
    # Stop condition
    if iter_num >= max_iter:
        break

In [29]:
weighted_predictions = []

for p,w_i in zip(adaboost_model['weak_predictors'], adaboost_model['weights']):
    x_test_array = np.array(x_test)
    predictions = p.predict(x_test_array)
    predictions = predictions * w_i #Multiply predictions by weigths of this perceptron
#     predictions = np.where(predictions == 0, -1, predictions) * w_i 
    weighted_predictions.append(predictions) #Append array of weighted predictions to the container


weighted_predictions = np.array(weighted_predictions)
# print(weighted_predictions.shape)

weighted_predictions = weighted_predictions.transpose() #Transpose array
# print(weighted_predictions.shape) 

final_prediction = []

for i in weighted_predictions:
    if i.mean() < 0: #Calcualte average of weighted prediction for each sample and compare with 0
        final_prediction.append(0) 
    else:
        final_prediction.append(1)

correct_count = (np.array(final_prediction) == np.array(y_test)).sum()
custom_aboost_accuracy = round((correct_count / len(y_test)*100), 2)
print(f"Custom Adabost with Perceptron accuracy  {custom_aboost_accuracy}%") 

Custom Adabost with Perceptron accuracy  91.63%


In [30]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier()
dtc.fit(x_train, y_train)
dtc_prediction = dtc.predict(x_test)
dtc_correct_count = (dtc_prediction == np.array(y_test)).sum()

sklearn_dtc_accuracy = round((dtc_correct_count / len(y_test)*100), 3)
print(f"Sklearn Decision Tree Classifier accuracy {sklearn_dtc_accuracy}%") 

Sklearn Decision Tree Classifier accuracy 82.433%


In [31]:
from sklearn.ensemble import AdaBoostClassifier

abc = AdaBoostClassifier()
abc.fit(x_train, y_train)
abc_prediction = abc.predict(x_test)
abc_correct_count = (abc_prediction == np.array(y_test)).sum()

sklearn_abc_accuracy = round((abc_correct_count / len(y_test)*100), 3)
print(f"Sklearn Adabost Classifier accuracy {sklearn_abc_accuracy}%") 

Sklearn Adabost Classifier accuracy 91.567%


In [32]:
from sklearn.metrics import accuracy_score

In [35]:
from sklearn.metrics import accuracy_score
models = ['My Decision Tree','My Adaboost with Perceptron', 'Sklearn Decision Tree Classifier', \
           'Sklearn Random Forest Classifier', 'Sklearn Adaboost Classifier']

compare_models = {'Model': models, \
                  'Accuracy': [custom_dt_accuracy, custom_aboost_accuracy, sklearn_dtc_accuracy, \
                               sklearn_rfc_accuracy, sklearn_abc_accuracy]}

models_df = pd.DataFrame(data = compare_models)

# print(models_df)

NameError: name 'sklearn_rfc_accuracy' is not defined

In [1]:
import matplotlib.pyplot as plt
order = [4, 3, 2, 1, 0]

y = [models[i] for i in order]
x = compare_models['Accuracy']
x = [x[i] for i in order]

plt.barh(y, x)

plt.xlabel("Accuracy %") 

plt.xlim([80, 100])
for i, v in enumerate(x):
    plt.text(v + .5, i , str(v)+'%')
    
plt.show()


NameError: name 'models' is not defined